In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/drive/MyDrive/KC House Price Prediction/df_final.csv')

In [4]:
df.head()

,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,month
0,0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650,2014,10
1,1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639,2014,12
2,2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062,2015,2
3,3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000,2014,12
4,4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503,2015,2


In [5]:
df.drop('Unnamed: 0', axis =1, inplace=True)

In [6]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,month
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650,2014,10
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639,2014,12
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062,2015,2
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000,2014,12
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503,2015,2


In [7]:
df.shape

(21597, 20)

In [8]:
x = df.drop('price', axis = 1)

In [9]:
y = df['price']

In [10]:
x.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,month
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650,2014,10
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639,2014,12
2,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062,2015,2
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000,2014,12
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503,2015,2


In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state = 1)

In [12]:
x_train.shape

(15117, 19)

In [13]:
y_train.shape

(15117,)

In [14]:
x_test.shape

(6480, 19)

In [15]:
y_test.shape

(6480,)

# **Feature importance and Model Training**

In [16]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,month
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650,2014,10
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639,2014,12
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062,2015,2
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000,2014,12
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503,2015,2


In [17]:
corr_df = pd.DataFrame(df[df.columns].corr()['price'][:])
corr_df = corr_df.reset_index()
corr_df.columns = ['columns', 'corr']
corr_df = corr_df.sort_values(by='corr', ascending=False)
corr_df = corr_df.iloc[1:, :]
corr_df = corr_df.reset_index(drop=True)
corr_df

,columns,corr
0,sqft_living,0.701917
1,grade,0.667951
2,sqft_above,0.605368
3,sqft_living15,0.585241
4,bathrooms,0.525906
5,view,0.397370
6,sqft_basement,0.323799
7,bedrooms,0.308787
8,lat,0.306692
9,waterfront,0.266398


In [18]:
def feature_importance_and_training(
    model_name = None,
    df = None,
    train_data_ = None,
    target_data_ = None,
    x_test = None,
    y_test = None,
    min_features = int,
    max_features = int,
    dictionary = {
                  'num_features' : [],
                  'score' : []
                  },
    ranked_feature_list = list(corr_df['columns'])
    ):

  for i in range(min_features, max_features+1):
    train_data = train_data_.copy()
    target_data = target_data_.copy()
  
    ## model and data definition
    if train_data is not None:
      train_data.reset_index(drop=True, inplace=True)
      print(train_data.shape)
      print(train_data.head())
    if target_data is not None:
      target_data.reset_index(drop=True, inplace=True)
      print(target_data.shape)
      print(target_data.head())
      print(ranked_feature_list[0:i])
      train_data = train_data[ranked_feature_list[0:i]]
      print(train_data.head())


    model = model_name
    model.fit(train_data, target_data)
    model_score = model.score(x_test[ranked_feature_list[0:i]], y_test)
    print(f'features : {ranked_feature_list[0:i]}')
    print(model_score)

    dictionary['num_features'].append(ranked_feature_list[0:i])
    dictionary['score'].append(model_score)

  return dictionary

#RandomForestRegressor

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
dictionary_score_rf = feature_importance_and_training(model_name = RandomForestRegressor(n_jobs = -1, random_state = 1),
                                df = df,
                                train_data_ = x_train,
                                target_data_ = y_train,
                                x_test = x_test,
                                y_test = y_test,
                                min_features = 3,
                                max_features = len(list(corr_df['columns'])),
                                dictionary = {
                                     'num_features' : [],
                                     'score' : []
                                      })

(15117, 19)
   bedrooms  bathrooms  sqft_living  ...  sqft_lot15  year  month
0         2       2.00          970  ...        9000  2014      7
1         3       1.00          940  ...        7980  2014     11
2         3       2.00         1290  ...        3235  2014      8
3         4       3.25         5640  ...       35033  2015      5
4         3       1.50         1230  ...        7800  2014     10

[5 rows x 19 columns]
(15117,)
0     303000.0
1     223000.0
2     450000.0
3    2000000.0
4     270000.0
Name: price, dtype: float64
['sqft_living', 'grade', 'sqft_above']
   sqft_living  grade  sqft_above
0          970      6         970
1          940      6         940
2         1290      8        1290
3         5640     11        4900
4         1230      7        1230
features : ['sqft_living', 'grade', 'sqft_above']
0.5075931139629295
(15117, 19)
   bedrooms  bathrooms  sqft_living  ...  sqft_lot15  year  month
0         2       2.00          970  ...        9000  2014      7
1

In [21]:
dictionary_score_rf 

{'num_features': [['sqft_living', 'grade', 'sqft_above'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms', 'view'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat',
   'waterfront'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat',
   'waterfront',
   'floors'],
  ['sqft_living',
 

In [22]:
df_rf = pd.DataFrame(dictionary_score_rf)
df_rf

,num_features,score
0,"[sqft_living, grade, sqft_above]",0.507593
1,"[sqft_living, grade, sqft_above, sqft_living15]",0.543749
2,"[sqft_living, grade, sqft_above, sqft_living15...",0.560043
3,"[sqft_living, grade, sqft_above, sqft_living15...",0.602951
4,"[sqft_living, grade, sqft_above, sqft_living15...",0.602520
5,"[sqft_living, grade, sqft_above, sqft_living15...",0.611159
6,"[sqft_living, grade, sqft_above, sqft_living15...",0.766106
7,"[sqft_living, grade, sqft_above, sqft_living15...",0.777058
8,"[sqft_living, grade, sqft_above, sqft_living15...",0.781614
9,"[sqft_living, grade, sqft_above, sqft_living15...",0.785187


In [23]:
df_rf.sort_values('score', ascending= False).reset_index(drop=True)['num_features']

0     [sqft_living, grade, sqft_above, sqft_living15...
1     [sqft_living, grade, sqft_above, sqft_living15...
2     [sqft_living, grade, sqft_above, sqft_living15...
3     [sqft_living, grade, sqft_above, sqft_living15...
4     [sqft_living, grade, sqft_above, sqft_living15...
5     [sqft_living, grade, sqft_above, sqft_living15...
6     [sqft_living, grade, sqft_above, sqft_living15...
7     [sqft_living, grade, sqft_above, sqft_living15...
8     [sqft_living, grade, sqft_above, sqft_living15...
9     [sqft_living, grade, sqft_above, sqft_living15...
10    [sqft_living, grade, sqft_above, sqft_living15...
11    [sqft_living, grade, sqft_above, sqft_living15...
12    [sqft_living, grade, sqft_above, sqft_living15...
13    [sqft_living, grade, sqft_above, sqft_living15...
14    [sqft_living, grade, sqft_above, sqft_living15...
15      [sqft_living, grade, sqft_above, sqft_living15]
16                     [sqft_living, grade, sqft_above]
Name: num_features, dtype: object

In [24]:
## selecting fetures which give maximum model score and after that tune model corresponding to that features

cols_rf = df_rf.sort_values('score', ascending= False).reset_index(drop=True)['num_features'][0]
cols_rf

['sqft_living',
 'grade',
 'sqft_above',
 'sqft_living15',
 'bathrooms',
 'view',
 'sqft_basement',
 'bedrooms',
 'lat',
 'waterfront',
 'floors',
 'yr_renovated',
 'sqft_lot',
 'sqft_lot15',
 'yr_built',
 'condition',
 'long',
 'year']

In [25]:
x_train_rf = x_train[cols_rf]
x_test_rf = x_test[cols_rf]

x_train_rf.head()

,sqft_living,grade,sqft_above,sqft_living15,bathrooms,view,sqft_basement,bedrooms,lat,waterfront,floors,yr_renovated,sqft_lot,sqft_lot15,yr_built,condition,long,year
8093,970,6,970,1850,2.00,0,0,2,47.5073,0,1.0,0,9750,9000,1940,5,-122.372,2014
13884,940,6,940,1050,1.00,0,0,3,47.5107,0,1.0,0,7980,7980,1961,3,-122.345,2014
406,1290,8,1290,1290,2.00,0,0,3,47.6609,0,3.0,0,1213,3235,2001,3,-122.344,2014
21317,5640,11,4900,4920,3.25,2,740,4,47.5491,0,2.0,0,35006,35033,2015,3,-122.104,2015
7766,1230,7,1230,1260,1.50,0,0,3,47.4202,0,1.0,0,7500,7800,1962,3,-122.331,2014


In [26]:
x_train_rf.shape

(15117, 18)

###RandomForestRegressor fine-tunning

In [27]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [28]:
rf_grid = {"n_estimators": [100, 200, 300, 400, 500],
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": [2, 5, 10],
           "min_samples_leaf": [1, 2, 4],
           "max_features": [0.5, 1, "sqrt", "auto"]}


rf_model = RandomizedSearchCV(RandomForestRegressor(),
                             param_distributions = rf_grid,
                             scoring = 'neg_mean_absolute_error', 
                             n_iter = 20,
                             cv = 5,
                             n_jobs=-1,
                             random_state = 1)

In [29]:
rf_model.fit(x_train_rf, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [30]:
y_pred_rf = rf_model.predict(x_test_rf)

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print('MAE:', mean_absolute_error(y_test, y_pred_rf))
print('MSE:', mean_squared_error(y_test, y_pred_rf))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R-squared score:", r2_score(y_test, y_pred_rf))

MAE: 69115.63485263214
MSE: 16485435443.318863
RMSE: 128395.62081052011
R-squared score: 0.8688127402530751


In [32]:
# best model hyperparameters
rf_model.best_params_

{'max_depth': None,
 'max_features': 0.5,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 400}

#XGBRegressor

In [33]:
from xgboost import XGBRegressor

In [34]:
dictionary_score_xgb = feature_importance_and_training(model_name = XGBRegressor(n_jobs=-1,random_state = 1),
                                df = df,
                                train_data_ = x_train,
                                target_data_ = y_train,
                                x_test = x_test,
                                y_test = y_test,
                                min_features = 3,
                                max_features = len(list(corr_df['columns'])),
                                dictionary = {
                                  'num_features' : [],
                                  'score' : [],
                                   })

(15117, 19)
   bedrooms  bathrooms  sqft_living  ...  sqft_lot15  year  month
0         2       2.00          970  ...        9000  2014      7
1         3       1.00          940  ...        7980  2014     11
2         3       2.00         1290  ...        3235  2014      8
3         4       3.25         5640  ...       35033  2015      5
4         3       1.50         1230  ...        7800  2014     10

[5 rows x 19 columns]
(15117,)
0     303000.0
1     223000.0
2     450000.0
3    2000000.0
4     270000.0
Name: price, dtype: float64
['sqft_living', 'grade', 'sqft_above']
   sqft_living  grade  sqft_above
0          970      6         970
1          940      6         940
2         1290      8        1290
3         5640     11        4900
4         1230      7        1230
[13:40:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
features : ['sqft_living', 'grade', 'sqft_above']
0.5662393108548389
(15117, 19)
   bed

In [35]:
dictionary_score_xgb

{'num_features': [['sqft_living', 'grade', 'sqft_above'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms', 'view'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat',
   'waterfront'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat',
   'waterfront',
   'floors'],
  ['sqft_living',
 

In [36]:
df_xgb = pd.DataFrame(dictionary_score_xgb)
df_xgb

,num_features,score
0,"[sqft_living, grade, sqft_above]",0.566239
1,"[sqft_living, grade, sqft_above, sqft_living15]",0.587721
2,"[sqft_living, grade, sqft_above, sqft_living15...",0.594912
3,"[sqft_living, grade, sqft_above, sqft_living15...",0.623937
4,"[sqft_living, grade, sqft_above, sqft_living15...",0.626633
5,"[sqft_living, grade, sqft_above, sqft_living15...",0.630201
6,"[sqft_living, grade, sqft_above, sqft_living15...",0.778338
7,"[sqft_living, grade, sqft_above, sqft_living15...",0.793692
8,"[sqft_living, grade, sqft_above, sqft_living15...",0.795594
9,"[sqft_living, grade, sqft_above, sqft_living15...",0.803657


In [37]:
## selecting fetures which give maximum model score and after that tune model corresponding to that features

cols_xgb = df_xgb.sort_values('score', ascending= False).reset_index(drop=True)['num_features'][0]
cols_xgb

['sqft_living',
 'grade',
 'sqft_above',
 'sqft_living15',
 'bathrooms',
 'view',
 'sqft_basement',
 'bedrooms',
 'lat',
 'waterfront',
 'floors',
 'yr_renovated',
 'sqft_lot',
 'sqft_lot15',
 'yr_built',
 'condition',
 'long',
 'year']

In [38]:
x_train_xgb = x_train[cols_xgb]
x_test_xgb = x_test[cols_xgb]

x_train_xgb.shape

(15117, 18)

###XGBRegressor fine-tunning

In [39]:
xgb_params = {'max_depth': [1, 2, 3, 4, 5],
              'learning_rate': [0.01, 0.05, 0.1],
              'n_estimators': [100, 200, 300, 400, 500],
              'reg_lambda': [0.001, 0.1, 1.0, 10.0, 100.0]}

xgb_model = RandomizedSearchCV(XGBRegressor(),
                                            xgb_params,
                                            n_iter=20,
                                            scoring = 'neg_mean_absolute_error',
                                            cv=5,
                                            n_jobs=-1,
                                            random_state = 1)

xgb_model.fit(x_train_xgb, y_train)

[13:44:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=...
                                          seed=None, silent=None, subsample=1,
                                          verbosity=1),
                   iid='deprecated', n_iter=20, n_job

In [40]:
y_pred_xgb = xgb_model.predict(x_test_xgb)

In [41]:
print('MAE:', mean_absolute_error(y_test, y_pred_xgb))
print('MSE:', mean_squared_error(y_test, y_pred_xgb))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print("R-squared score:", r2_score(y_test, y_pred_xgb))

MAE: 64405.91650993441
MSE: 12395531673.783403
RMSE: 111335.22207182866
R-squared score: 0.9013592428916465


In [42]:
# best model hyperparameters
xgb_model.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500, 'reg_lambda': 10.0}

#KNeighborsRegressor

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [44]:
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.transform(x_test)

In [45]:
scaled_x_train = pd.DataFrame(scaled_x_train, columns = x_train.columns)

In [46]:
scaled_x_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,month
0,-1.470325,-0.157060,-1.214613,-0.127844,-0.919344,-0.087936,-0.310091,2.440070,-1.413563,-0.990298,-0.662635,-1.049996,-0.211107,-0.382002,-1.119303,-0.203735,-0.139025,-0.691586,0.132454
1,-0.402135,-1.454832,-1.247371,-0.170023,-0.919344,-0.087936,-0.310091,-0.629204,-1.413563,-1.026510,-0.662635,-0.337656,-0.211107,-0.357542,-0.926996,-1.366113,-0.176082,-0.691586,1.413315
2,-0.402135,-0.157060,-0.865198,-0.331278,2.782273,-0.087936,-0.310091,-0.629204,0.285950,-0.604041,-0.662635,1.019180,-0.211107,0.723036,-0.919873,-1.017399,-0.348472,-0.691586,0.452669
3,0.666056,1.465155,3.884666,0.473996,0.931465,-0.087936,2.279771,-0.629204,2.835220,3.753425,1.017013,1.494073,-0.211107,-0.081282,0.789529,4.256893,0.806776,1.445953,-0.507977
4,-0.402135,-0.805946,-0.930713,-0.181461,-0.919344,-0.087936,-0.310091,-0.629204,-0.563806,-0.676464,-0.662635,-0.303735,-0.211107,-1.008623,-0.827281,-1.060989,-0.182622,-0.691586,1.093100


In [47]:
scaled_x_test = pd.DataFrame(scaled_x_test, columns = x_test.columns)

In [48]:
from sklearn.neighbors import KNeighborsRegressor

In [49]:
dictionary_score_knn = feature_importance_and_training(model_name = KNeighborsRegressor(),
                                                   df = df,
                                train_data_ = scaled_x_train,
                                target_data_ = y_train,
                                x_test = scaled_x_test ,
                                y_test = y_test,
                                min_features = 3,
                                max_features = len(list(corr_df['columns'])),
                                dictionary = {
                                  'num_features' : [],
                                  'score' : [],
                                   })

(15117, 19)
   bedrooms  bathrooms  sqft_living  ...  sqft_lot15      year     month
0 -1.470325  -0.157060    -1.214613  ...   -0.139025 -0.691586  0.132454
1 -0.402135  -1.454832    -1.247371  ...   -0.176082 -0.691586  1.413315
2 -0.402135  -0.157060    -0.865198  ...   -0.348472 -0.691586  0.452669
3  0.666056   1.465155     3.884666  ...    0.806776  1.445953 -0.507977
4 -0.402135  -0.805946    -0.930713  ...   -0.182622 -0.691586  1.093100

[5 rows x 19 columns]
(15117,)
0     303000.0
1     223000.0
2     450000.0
3    2000000.0
4     270000.0
Name: price, dtype: float64
['sqft_living', 'grade', 'sqft_above']
   sqft_living     grade  sqft_above
0    -1.214613 -1.413563   -0.990298
1    -1.247371 -1.413563   -1.026510
2    -0.865198  0.285950   -0.604041
3     3.884666  2.835220    3.753425
4    -0.930713 -0.563806   -0.676464
features : ['sqft_living', 'grade', 'sqft_above']
0.5206844861445
(15117, 19)
   bedrooms  bathrooms  sqft_living  ...  sqft_lot15      year     month
0 -

In [50]:
dictionary_score_knn

{'num_features': [['sqft_living', 'grade', 'sqft_above'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms'],
  ['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms', 'view'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat',
   'waterfront'],
  ['sqft_living',
   'grade',
   'sqft_above',
   'sqft_living15',
   'bathrooms',
   'view',
   'sqft_basement',
   'bedrooms',
   'lat',
   'waterfront',
   'floors'],
  ['sqft_living',
 

In [51]:
df_knn = pd.DataFrame(dictionary_score_knn)
df_knn

,num_features,score
0,"[sqft_living, grade, sqft_above]",0.520684
1,"[sqft_living, grade, sqft_above, sqft_living15]",0.540519
2,"[sqft_living, grade, sqft_above, sqft_living15...",0.551568
3,"[sqft_living, grade, sqft_above, sqft_living15...",0.591302
4,"[sqft_living, grade, sqft_above, sqft_living15...",0.588931
5,"[sqft_living, grade, sqft_above, sqft_living15...",0.596888
6,"[sqft_living, grade, sqft_above, sqft_living15...",0.728245
7,"[sqft_living, grade, sqft_above, sqft_living15...",0.751084
8,"[sqft_living, grade, sqft_above, sqft_living15...",0.758946
9,"[sqft_living, grade, sqft_above, sqft_living15...",0.759865


In [52]:
## selecting fetures which give maximum model score and after that tune model corresponding to that features

cols_knn = df_knn.sort_values('score', ascending= False).reset_index(drop=True)['num_features'][0]
cols_knn

['sqft_living',
 'grade',
 'sqft_above',
 'sqft_living15',
 'bathrooms',
 'view',
 'sqft_basement',
 'bedrooms',
 'lat',
 'waterfront',
 'floors',
 'yr_renovated',
 'sqft_lot',
 'sqft_lot15',
 'yr_built',
 'condition',
 'long']

In [53]:
scaled_x_train_knn = scaled_x_train[cols_knn]
scaled_x_test_knn = scaled_x_test[cols_knn]
x_train_xgb.shape

(15117, 18)

###KNeighborsRegressor fine-tunning

In [54]:
k_range = list(range(1, 30))
params = dict(n_neighbors = k_range)
knn_regressor = GridSearchCV(KNeighborsRegressor(), params, cv = 10, scoring = 'neg_mean_squared_error')
knn_regressor.fit(scaled_x_train_knn, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [55]:
y_pred_kneighbor = knn_regressor.predict(scaled_x_test_knn)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('MAE:', mean_absolute_error(y_test, y_pred_kneighbor))
print('MSE:', mean_squared_error(y_test, y_pred_kneighbor))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_kneighbor)))
print("R-squared score:", r2_score(y_test, y_pred_kneighbor))

MAE: 89896.66655643741
MSE: 25852651824.071915
RMSE: 160787.5984772206
R-squared score: 0.7942706116770583


In [56]:
# best model hyperparameters
knn_regressor.best_params_

{'n_neighbors': 7}